In [ ]:
'''
This script takes the csv files  and filters out the words which are not present in UMLS.
'''

import argparse
from quickumls import QuickUMLS
import pandas as pd
import glob


umls_path = '/path/to/umls'
matcher = QuickUMLS(umls_path,similarity_name='cosine',threshold=0.95) 

In [ ]:
from collections import defaultdict
counter_PAC = defaultdict(int)
for csv_path in ['../../../../TxtInputFiles/PAC_input.txt']:

    print(f'Starting for {csv_path}.....')
    lines_PAC = open(csv_path).readlines()

    for idx,abs in enumerate(lines_PAC):
        if idx%100 == 0: print(f'Processed till {idx+1}... {len(counter_PAC)} are considered till now.')

        flag = 0
        d = matcher.match(abs, best_match=True, ignore_syntax=False)
        if len(d) == 0:
            continue
        
        else:
            for l in d:
                counter_PAC[l[0]['ngram']] += 1
                
    if idx%10000 == 0: print(f'Processed till {idx+1}... {len(counter_PAC)} are considered till now.')

    # df.to_csv('./%sWithConsiderFlag_AllSemTypes.csv'%csv_path.split('/')[-1][:-4],index=False)

In [ ]:
import pandas as pd

list_tokens = list(counter_PAC.keys())
val_tokens = list(counter_PAC.values())


In [ ]:
df = pd.DataFrame({'Tokens':list_tokens,'Count':val_tokens})
df.to_csv('PAC_Tokens.csv',index=False)

In [ ]:
import numpy as np
np.percentile(val_tokens,[0,10,25,50,75,90,95,90,100])

In [ ]:
from collections import defaultdict
import json
counter_PAC = defaultdict(int)
for csv_path in ['./EBM_Train.jsonl']:

    print(f'Starting for {csv_path}.....')
    lines_PAC = open(csv_path).readlines()
    lines_PAC = [json.loads(l)['prompt'] for l in lines_PAC]

    for idx,abs in enumerate(lines_PAC):
        if idx%100 == 0: print(f'Processed till {idx+1}... {len(counter_PAC)} are considered till now.')

        flag = 0
        d = matcher.match(abs, best_match=True, ignore_syntax=False)
        if len(d) == 0:
            continue
        
        else:
            for l in d:
                counter_PAC[l[0]['ngram']] += 1
                
    if idx%100 == 0: print(f'Processed till {idx+1}... {len(counter_PAC)} are considered till now.')

    # df.to_csv('./%sWithConsiderFlag_AllSemTypes.csv'%csv_path.split('/')[-1][:-4],index=False)

In [ ]:
import pandas as pd

list_tokens = list(counter_PAC.keys())
val_tokens = list(counter_PAC.values())

df = pd.DataFrame({'Tokens':list_tokens,'Count':val_tokens})
df.to_csv('EBM_Tokens.csv',index=False)

In [1]:
import pandas as pd
df_PAC = pd.read_csv('PAC_Tokens.csv')
df_BioASQ = pd.read_csv('EBM_Tokens.csv')

df_BioASQ

,Tokens,Count
0,previous myocardial infarction,2
1,high-density lipoprotein,27
2,low-density lipoprotein,35
3,Coronary heart disease,2
4,cardiovascular events,37
...,...,...
12335,band counts,1
12336,cell counts,2
12337,lidocaine-prilocaine,1
12338,topical anaesthetic,1


In [2]:
df_BioASQ['Count'].describe()

count    12340.000000
mean        13.448703
std         87.294873
min          1.000000
25%          1.000000
50%          2.000000
75%          6.000000
max       4269.000000
Name: Count, dtype: float64

In [3]:
from collections import defaultdict
tokens_PAC = df_PAC.set_index('Tokens')['Count'].to_dict()
tokens_PAC_words = defaultdict(int)

tokens_BioASQ = df_BioASQ.set_index('Tokens')['Count'].to_dict()
tokens_BioASQ_words = defaultdict(int)

import re
pattern = r"^[A-Za-z▁]+$"

for key,val in tokens_PAC.items():
    try:
      for word in key.split():  
        if re.match(pattern,word.strip()):
            tokens_PAC_words[word] += val
    except: pass
  
for key,val in tokens_BioASQ.items():
    try:
      for word in key.split():  
        if re.match(pattern,word.strip()):
            tokens_BioASQ_words[word] += val
    except: pass


In [4]:
df_tokens_PAC_words = pd.DataFrame(list(tokens_PAC_words.items()), columns=['Word', 'Count'])
print(df_tokens_PAC_words)

df_BioASQ_words = pd.DataFrame(list(tokens_BioASQ_words.items()), columns=['Word', 'Count'])
print(df_BioASQ_words)

                     Word  Count
0                Yersinia    387
1          enterocolitica    155
2           agglutination   1261
3                reaction  30928
4              rheumatoid   2403
...                   ...    ...
81111  choledochoduodenal      1
81112           mabuterol      3
81113              Varroa      1
81114           jacobsoni      1
81115           anopheles      1

[81116 rows x 2 columns]
             Word  Count
0        previous    108
1      myocardial    128
2      infarction    116
3     lipoprotein    105
4        Coronary      5
...           ...    ...
8686   clustering      1
8687    offspring      4
8688      origins      1
8689        Today      1
8690          CBC      1

[8691 rows x 2 columns]


In [5]:
top_50_percent_PAC = df_tokens_PAC_words.nlargest(int(len(df_tokens_PAC_words)), 'Count')
top_50_percent_BioASQ = df_BioASQ_words.nlargest(int(len(df_BioASQ_words)), 'Count')

print(top_50_percent_PAC)
print(top_50_percent_BioASQ)

               Word   Count
240           cells  406536
29         patients  393424
262            cell  221685
106            less  204305
438        activity  184463
...             ...     ...
66178  textilotoxin       1
66177   ammodytoxin       1
47236    Converting       1
66174         Brant       1
81115     anopheles       1

[81116 rows x 2 columns]
              Word  Count
58           query   4269
37        patients   3291
23     information   2993
72             You   2846
146      treatment   2166
...            ...    ...
4046   orthopaedic      1
4045  orthopaedics      1
6797          four      1
6798     quadrants      1
8690           CBC      1

[8691 rows x 2 columns]


In [6]:
from transformers import AutoTokenizer
tokenier = AutoTokenizer.from_pretrained('meta-llama/Llama-3.1-8B')
pre_vocab = tokenier.get_vocab()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
df_common = top_50_percent_PAC[top_50_percent_PAC['Word'].isin(top_50_percent_BioASQ['Word'])]
df_common['Splits'] = df_common['Word'].apply(lambda x: len(tokenier.tokenize(x)))


/var/folders/p2/b5tg3bsn4l5cnf3dy16wx6fh0000gn/T/ipykernel_13204/1201604217.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_common['Splits'] = df_common['Word'].apply(lambda x: len(tokenier.tokenize(x)))


In [8]:
lookup_table_1 = df_common[df_common['Splits'] > 1]
lookup_table_1.to_csv('EBM_SplitMoreThan1_OOV.csv',index=False)

In [9]:
import pandas as pd
import os

lookup_table_1 = pd.read_csv('./EBM_SplitMoreThan1_OOV.csv')
lookup_table_words_1 = lookup_table_1['Word'].to_list()

In [10]:
from collections import defaultdict

os.mkdir('./EBM_Lookup_SplitMoreThan1_Vocab')


for i in range(50,501,50):
    dump_words = []
    for word in lookup_table_words_1:
        if len(dump_words) == i:
            break
        
        if word not in pre_vocab:
            dump_words.append(word)
        
    with open(f'EBM_Lookup_SplitMoreThan1_Vocab/EBM_Lookup_{i}.txt','w') as f: #{i+500}.txt','w') as f:
        f.write('\n'.join([f'''{x}''' for idx,x in enumerate(dump_words)]))
        f.write('\n')
    f.close()

In [ ]:
org_tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-3.1-8B')
vocab_tokenizer = AutoTokenizer.from_pretrained('./EBM_Lookup_SplitMoreThan1_Vocab_Llama3/EBM_Lookup_500')
idx = 0
with open('../../../../../TxtInputFiles/PAC_input.txt') as f:
    for line in f:
        line = line.strip()
        org_dec = org_tokenizer.decode(org_tokenizer.encode(line))
        vocab_dec = vocab_tokenizer.decode(vocab_tokenizer.encode(line))
        
        assert org_dec == vocab_dec, f'Failed at {idx}'
        
        idx += 1
        if idx%1000 == 0:
            print(f'Processed {idx}....')
        